In [1]:
import gym
import ptan
from typing import List, Optional, Tuple, Any

In [3]:
class ToyEnv(gym.Env):
    """
    Environment with observation 0..4 and actions 0..2
    Observations are roated sequentially mod 5, reward is equal to given action.
    Episodes are having fixed length of 10
    """
    
    def __init__(self):
        super(ToyEnv, self).__init__()
        self.observation_space = gym.spaces.Discrete(n=5)
        self.action_space = gym.spaces.Discrete(n=3)
        self.step_index = 0
        
    def reset(self):
        self.step_index = 0
        return self.step_index
    
    def step(self, action):
        is_done = self.step_index == 10
        if is_done:
            return self.step_index % self.observation_space.n, 0.0, is_done, {}
        self.step_index += 1
        return self.step_index % self.observation_space.n, float(action), self.step_index == 10, {}

In [4]:
class DullAgent(ptan.agent.BaseAgent):
    """ 
    Agent always returns the fixed return
    """
    
    def __init__(self, action: int):
        self.action = action
        
    def __call__(self, observations: List[Any], state: Optional[List] = None) -> Tuple[List[int], Optional[List]]:
        return [self.action for _ in observations], state

In [6]:
env = ToyEnv()
agent = DullAgent(action=1)
exp_source = ptan.experience.ExperienceSourceFirstLast(env, agent, gamma=1.0, steps_count=1)
buffer = ptan.experience.ExperienceReplayBuffer(exp_source, buffer_size=100)

for step in range(6):
    buffer.populate(1)
    # if buffer is small enough, do nothing
    if len(buffer) < 5:
        continue
    batch = buffer.sample(4)
    print("Train time, %d batch size: " % len(batch))
    for s in batch:
        print(s)

Train time, 4 batch size: 
ExperienceFirstLast(state=4, action=1, reward=1.0, last_state=0)
ExperienceFirstLast(state=1, action=1, reward=1.0, last_state=2)
ExperienceFirstLast(state=0, action=1, reward=1.0, last_state=1)
ExperienceFirstLast(state=2, action=1, reward=1.0, last_state=3)
Train time, 4 batch size: 
ExperienceFirstLast(state=3, action=1, reward=1.0, last_state=4)
ExperienceFirstLast(state=4, action=1, reward=1.0, last_state=0)
ExperienceFirstLast(state=2, action=1, reward=1.0, last_state=3)
ExperienceFirstLast(state=0, action=1, reward=1.0, last_state=1)
